# Ingest catalogs health check results
Using spark functions health checks are done on catalogs to see if they are available 
from the context of the notebook run as login
a runID and timestamp columns were added so that each run of this notebook results are uniquely
captured

In [0]:
from pyspark.sql import functions as F
import uuid
from datetime import datetime, timezone

#  Generate run ID mask to 63 bits
runid_full = uuid.uuid4().int
runid      = int(runid_full & ((1 << 63) - 1))
ts         = datetime.now(timezone.utc)

#  Grab the notebook context
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()

# Extract the workspace-specific URL
def get_workspace_url(workspace_id):
    return f"https://adb-{workspace_id}.2.azuredatabricks.net"

# Get the workspace ID
workspace_id = ctx.workspaceId().get()
workspace_name = get_workspace_url(workspace_id)

https://adb-2123484989296422.2.azuredatabricks.net


In [0]:
#  Attempt to fetch list of catalogs
try:
    catalogs_df = spark.sql("SHOW CATALOGS")
    catalogs_df = catalogs_df.withColumnRenamed("catalog", "checkname")   
    catalogs_df = catalogs_df.withColumn(
        "checkname",
        F.concat(
            F.lit("checking catalog "),
            F.col("checkname")
        )
    )
    status, level, msg = "healthy", "info", "Fetched catalog successfully"
    count = catalogs_df.count()
except Exception as e:
    # If Unity Catalog not enabled, fallback or record the error
    status, level, msg = "unhealthy", "critical", str(e)
    catalogs_df = spark.createDataFrame([], schema="checkname string")
    count = 0

In [0]:
from pyspark.sql.functions import parse_json

# Enrich fields
health_df = (
    catalogs_df 
    .withColumn("runid", F.lit(runid).cast("long"))
    .withColumn("timestamp", F.lit(ts).cast("timestamp"))
    .withColumn("status", F.lit(status))
    .withColumn("level", F.lit(level))
    .withColumn("message", F.lit(msg))
    .withColumn("extended_properties", parse_json(
                    F.to_json(
                        F.struct(
                            F.lit(count).alias("catalog_count"),
                            F.lit(workspace_name).alias("workspace")
                        )
                    )
                )
    )
)

In [0]:
#  Enable Delta schema merging
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

In [0]:
#  Append into delta table
health_df.write \
    .format("delta") \
    .mode("append") \
    .option("mergeSchema", "false") \
    .saveAsTable("platform.health_check_results")

print("Write complete .")

Write complete .
